
Datenverteilung: {'1': 201688, '2': 151058, '3': 254295, '4': 585616, '5': 1807343}
    --> Max 150'000 pro Klasse um Ungleichgewicht zu verhindern

In [44]:
#In order to test with a subset. Don't execute for real run
import pandas as pd

data = pd.read_csv("data/Books_rating.csv", sep=",")
counts = {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0}
for index, row in data.iterrows():
    reviewScore = '{0:g}'.format(row["review/score"])
    counts[reviewScore] = counts[reviewScore] + 1
counts

{'1': 201688, '2': 151058, '3': 254295, '4': 585616, '5': 1807343}

In [2]:
from pathlib import Path
import pandas as pd

Path("D:/nlp/1").mkdir(parents=True, exist_ok=True)
Path("D:/nlp/2").mkdir(parents=True, exist_ok=True)
Path("D:/nlp/3").mkdir(parents=True, exist_ok=True)
Path("D:/nlp/4").mkdir(parents=True, exist_ok=True)
Path("D:/nlp/5").mkdir(parents=True, exist_ok=True)

counts = {"1": 0, "2": 0, "3": 0, "4": 0, "5": 0}

data = pd.read_csv("data/Books_rating.csv", sep=",")
for index, row in data.iterrows():
    reviewScore = '{0:g}'.format(row["review/score"])
    counts[reviewScore] = counts[reviewScore] + 1
    if (counts[reviewScore] > 150000):
        continue
    fp = open("D:/nlp/" + reviewScore + "/" + str(index) + ".txt", "w")
    fp.write(row["review/text"])
    fp.close()

In [1]:
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import re
import nltk
from sklearn.datasets import load_files

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

data = load_files(r"D:/nlp")
X, y = data.data, data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0)

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pasca\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pasca\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pasca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[[81617 22813 11901  5872 12793]
 [32481 53064 28295 10064 11078]
 [15957 21525 57809 23562 16029]
 [ 9875  8191 29561 49100 38296]
 [11239  4980 11175 25726 81997]]
              precision    recall  f1-score   support

           0       0.54      0.60      0.57    134996
           1       0.48      0.39      0.43    134982
           2       0.42      0.43      0.42    134882
           3       0.43      0.36      0.39    135023
           4       0.51      0.61      0.56    135117

    accuracy                           0.48    675000
   macro avg       0.48      0.48      0.47    675000
weighted avg       0.48      0.48      0.47    675000

